In [76]:
import requests

headers = {
    'authority': 'dce-gempak-api-prd.eco.astro.com.my',
    'accept': '*/*',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    'origin': 'https://gempak.com',
    'pragma': 'no-cache',
    'referer': 'https://gempak.com/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

params = {
    'type': 'article',
    'siteId': '1',
    'fields': 'playlistDetails',
    'pillarCategoryId': '227,1405,1438,1253',
    'pageSize': '100',
    'pageNumber': '1',
}

response = requests.get('https://dce-gempak-api-prd.eco.astro.com.my/v2/feed', params=params, headers=headers)

In [79]:
!mkdir url

In [81]:
from tqdm import tqdm
import json
import os

def generator():
    while True:
        yield


index = 1
for _ in tqdm(generator()):
    filename = os.path.join('url', f'{index}.json')
    if os.path.exists(filename):
        index += 1
        continue
    
    params = {
        'type': 'article',
        'siteId': '1',
        'fields': 'playlistDetails',
        'pillarCategoryId': '227,1405,1438,1253',
        'pageSize': '100',
        'pageNumber': str(index),
    }
    
    response = requests.get('https://dce-gempak-api-prd.eco.astro.com.my/v2/feed', params=params, headers=headers)
    r = response.json()
    if r['responseCode'] != 200:
        break
    
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)
    
    index += 1

100it [00:15,  6.46it/s]


In [1]:
import re

def get_title(row):
    title = row['title']
    id = row['id']
    string = re.sub('[^a-z0-9 ]+', '', title.lower())
    string = re.sub(r'[ ]+', ' ', string).strip().replace(' ', '-')
    return f'{string}-{id}'

# get_title(r['response'][1])

In [5]:
from glob import glob
import json

In [6]:
files = glob('url/*.json')
len(files)

100

In [7]:
urls = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
        
    for row in data['response']:
        urls.append('https://gempak.com/berita-terkini/' + get_title(row))

In [8]:
len(set(urls))

9999

In [9]:
with open('url.json', 'w') as fopen:
    json.dump(list(set(urls)), fopen)